# Modulos

In [1]:
import os
import re
import tabula
import pandas as pd
from tabula.io import read_pdf

os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-23'

# Funciones

In [2]:
def remove_footer(x):
    return re.sub(r'\s*\(\d+\)', '', x).strip() 

# Lectura de datos

In [3]:
folder= "PDFs/"
ls_files = [x for x in os.listdir(folder) if '.pdf' in x]

## Tasas de descuento por institución

In [4]:
ls_data=[]
for file in ls_files:
    tables = read_pdf(folder+file, pages="all", encoding='latin1',stream=True ) 
    if(len(tables)>1):
        aux= pd.concat(tables, ignore_index=True)
    else: 
        aux= tables[0]
    ls_values= [x for x in aux.columns if x !="Institución"]
    aux["Institución"] = aux["Institución"].apply(lambda x: remove_footer(x))
    aux[ls_values]= aux[ls_values].map(lambda x: float(x.replace("-","0").replace("%", "")))
    aux["Tipo"] = file.split(" ")[1]
    aux["Familia"] = " ".join(file.split(" ")[2:]).replace(".pdf", "") 
    ls_data.append(aux)

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [5]:
data= pd.concat(ls_data, ignore_index=True)
data.columns = ['Institucion', 'Promedio', 'Minima', 'Maxima', 'Desviación', 'Moda', 'Tipo', 'Familia']

In [6]:
data.Institucion.unique()

array(['EVO', 'Mifel', 'Dlocal', 'Getnet', 'Afirme', 'Banorte',
       'BBVA Bancomer', 'Autofin', 'Inbursa', 'Adyen', 'Banregio',
       'Multiva', 'Fiserv', 'Bansí', 'Azteca', 'Intercam Banco', 'Bajío',
       'Bzpay', 'Velpay', 'Mercado Pago', 'Espiral', 'Paypal',
       'PayU Digicuenta', 'Pocket', 'Pago Fácil', 'Sr. Pago', 'Stripe',
       'Clip', 'Smart Payment', 'Ebanx', 'Xaldo', 'Cyborgbeat', 'Adquira',
       'OEL', 'MerIT', 'EMS', 'Qiubo', 'Banjército'], dtype=object)

In [7]:
data[['Familia', 'Tipo','Institucion',  'Promedio', 'Minima', 'Maxima', 'Desviación', 'Moda' ]].sort_values(by=['Familia', 'Tipo']).to_csv("tasas_desc.csv", index=False)